In [2]:
import pandas as pd 
import numpy as np

dates = pd.date_range("2024-01-01", periods=6,freq='D')
tickers = ["AAPL","MSFT","NVDA"]
df = pd.MultiIndex.from_product([dates,tickers], names=['date','ticker']).to_frame(index=False)
sector_map = {"AAPL":"Tech", "MSFT":"Tech", "NVDA":"Semis"}
df['sector'] = df['ticker'].map(sector_map)

base = {"AAPL":180, "MSFT":400, "NVDA":500}
df['day_num'] = (df['date'] - df['date'].min()).dt.days
df['close'] = df.apply(lambda r : base[r['ticker']] + 2*r['day_num'] ,axis=1).astype(float)

df.loc[(df['ticker'] == 'AAPL') & (df['day_num'] == 3), 'close' ] = np.nan
df.loc[(df['ticker'] == 'MSFT') & (df['day_num'] == 4), 'close' ] = np.nan
rng = np.random.default_rng(0)
df["volume"]  = (rng.integers(100, 500, len(df)) * (1 + df["day_num"]*0.1)).round().astype(int)

df = df.sort_values(['date','ticker']).reset_index(drop=True)
print(df)

total_per_sector = df.groupby('sector')['volume'].sum()
multi_agg =df.groupby(['date','sector'], as_index=False).agg(
    rows=('close','size'),
    avg_close=('volume','mean')
)

df['mean_close'] = df.groupby('date')['close'].transform('mean')
df['normalized_close'] = df['close'] / df['mean_close']

grp = df.groupby('ticker')['close']
df['z-score'] = (df['close'] -  grp.transform('mean')) / grp.transform('std')

df['rolling_dev'] = df.groupby('ticker')['close'].transform(lambda s: s.rolling(window=3,min_periods=1).std())

df.sort_values('close',ascending=True).groupby('date').head(1)

# print(df[['ticker','z-score','date','rolling_dev','close']].head(9))
def _summary(g):
    return pd.DataFrame({
        "first_date":[g["date"].min()],
        "n_days":[g["date"].nunique()],
        "close_min":[g["close"].min()],
        "close_max":[g["close"].max()]
    })

per_sec_summ = df.groupby('sector').apply(_summary).reset_index(level=0)

df_idx = df.set_index(['date','ticker']).sort_index()
df_msft_closes = df_idx.xs("MSFT", level='ticker')['close']
df_idx['pct_change'] = df_idx.groupby(level='ticker')['close'].pct_change()
df_idx['roll_5_mean'] = df_idx.groupby(level='ticker')['close'
].transform(lambda d: d.rolling( window=5, min_periods=1 ).mean())
print(df_idx[["close","pct_change","roll_5_mean"]].head(9))

df_filtered = df_idx.groupby(level='ticker').filter(lambda g: g['close'].count() >= 6)

print(df_filtered.groupby(level='ticker')['close'].count())

df_dropped_sectors = df_idx.groupby('sector').filter(lambda d: d['volume'].mean() > 1500)
print(df_dropped_sectors.groupby('sector')['volume'].mean())

         date ticker sector  day_num  close  volume
0  2024-01-01   AAPL   Tech        0  180.0     440
1  2024-01-01   MSFT   Tech        0  400.0     354
2  2024-01-01   NVDA  Semis        0  500.0     304
3  2024-01-02   AAPL   Tech        1  182.0     228
4  2024-01-02   MSFT   Tech        1  402.0     245
5  2024-01-02   NVDA  Semis        1  502.0     128
6  2024-01-03   AAPL   Tech        2  184.0     156
7  2024-01-03   MSFT   Tech        2  404.0     127
8  2024-01-03   NVDA  Semis        2  504.0     204
9  2024-01-04   AAPL   Tech        3    NaN     552
10 2024-01-04   MSFT   Tech        3  406.0     467
11 2024-01-04   NVDA  Semis        3  506.0     604
12 2024-01-05   AAPL   Tech        4  188.0     421
13 2024-01-05   MSFT   Tech        4    NaN     479
14 2024-01-05   NVDA  Semis        4  508.0     683
15 2024-01-06   AAPL   Tech        5  190.0     586
16 2024-01-06   MSFT   Tech        5  410.0     528
17 2024-01-06   NVDA  Semis        5  510.0     476
            

In [ ]:
import pandas as pd
import numpy as np

date = pd.date_range('2024-01-01', periods=6, freq='D')
tickers = ["AAPL","MSFT","NVDA"]

df = pd.MultiIndex.from_product([date,tickers], names=['date','ticker']).to_frame(index=False)
sector_map = {"AAPL":"Tech", "MSFT":"Tech", "NVDA":"Semis"}

df['sector'] = df['ticker'].map(sector_map)

# Synthetic close prices
base = {"AAPL":180, "MSFT":400, "NVDA":500}
df['day_num'] = (df['date'] - df['date'].min()).dt.days
df['close'] = df.apply(lambda r: base[r['ticker']] + 2* r['day_num'],axis=1)

# Insert missing closes
df.loc[(df['ticker'] == 'AAPL') & (df['day_num'] == 3), 'close'] = np.nan
df.loc[(df['ticker'] == 'MSFT') & (df['day_num'] == 4), 'close'] = np.nan

# Synthetic volume
rng = np.random.default_rng(0)
df["volume"]  = (rng.integers(100, 500, len(df)) * (1 + df["day_num"]*0.1)).round().astype(int)

df = df.sort_values(['date','ticker']).reset_index(drop=True)

# print(df.isna().sum())
# print(df.groupby('ticker')['close'].mean())
# print(df['volume'].sum())

df['close_filled'] = df.groupby('ticker')['close'].ffill()
df['close_backfilled'] = df.groupby('ticker')['close'].bfill()
# print(df[['date','ticker','close','close_filled','close_backfilled']])
# print(df[df['close'].isna()])

# Keep only Tech sector rows where volume is above the overall median, sorted by date then -volume
mask = (df['sector'] == 'Tech') & (df['volume'] > df['volume'].median())
subset = df[mask].sort_values(['date','volume'], ascending=[True,False])
#Top-3 close per date (ties break by ticker).
print(df.sort_values(['date','close','ticker']).groupby('date').head(3)['close'])
#Lowest 2 volume per ticker.
df.sort_values(['ticker','volume'], ascending=[True,False]).groupby('ticker').head(2)
# Daily return per ticker
df['daily_ret'] = df.groupby('ticker')['close'].pct_change()
# Flag big_move if abs(ret) > ticker’s 75th percentile
df['big_move'] = df['daily_ret'].abs() > df.groupby('ticker')['daily_ret'].transform(lambda x: x.abs().quantile(0.75))
#Per-ticker close min, max, mean, std in one go.
df.groupby('ticker')['close'].agg(['min','max','mean','std'])
#Per-sector: rows = size(close), non_na = count(close), avg_close = mean(close), sum_vol = sum(volume)
df.groupby('sector').agg(
    rows=('close','size'),
    non_na= ('close','count'),
    avg_close=('close','mean'),
    sum_vol=('volume','sum')
)
#Per (date, sector): volume sum, median, 90th pct
#p90 = lambda s:s.quantile(0.9)
df.groupby(['date','sector'])['volume'].agg(['sum','median',p90 ])
#Which ticker has the highest average dollar_vol
df['dollar_volume'] = df['volume'] * df['close']
df.groupby('ticker')['dollar_volume'].agg('mean').idxmax()
#Fraction of rows per sector, sorted descending.
df.value_counts(normalize=True).sort_values(ascending=False)
#Build top-1 per day by close. which ticker most often wins? show proportion.
df['top-1c']=df.sort_values(['date','close'],ascending=False).groupby('date')['close'].head(1)
df['top-1c'].value_counts(normalize=True)
#Missing close AND volume in top quartile
msk = (df['close'].isna() ) & (df['volume'] >= df['volume'].quantile(.75))

#ret < 0 AND volume above its date’s median
msk = (df['daily_ret'] < 0) & (df['volume'] > df.groupby('date')['volume'].transform(lambda s: s.median())) 
#print(df[msk])
#For each date, z-score of close (cross-sectional : group by date, different entities at the same point in time)
df['z-score-xsec'] = df.groupby('date')['close'].transform(lambda s: (s-s.mean())/ s.std())
#For each ticker, z-score of ret (time-series)
df['z-score-times'] = df.groupby('ticker')['daily_ret'].transform(lambda s: (s-s.mean())/s.std())
#Per-sector custom summary DataFrame: first_date, last_date, max_close_ticker
def summary(g):
    return pd.DataFrame({
        "first_date":[g["date"].min()],
        "last_days":[g["date"].max()],
        "max_close_ticker":[g["close"].idxmax()]
    })
df.groupby('sector').apply(summary)
#Per-ticker MA(5) and Vol(5) on close (moving avg and volatility)    
grp= df.groupby('ticker')['close']
grp.rolling(5).mean()
grp.rolling(5).std()
#Per-ticker expanding cumulative return from first close
grp.pct_change().add(1).cumprod().sub(1)
#Per-ticker EWMA(10) of close (EWMA = Exponentially Weighted Moving Average)


0     180.0
1     400.0
2     500.0
3     182.0
4     402.0
5     502.0
6     184.0
7     404.0
8     504.0
10    406.0
11    506.0
9       NaN
12    188.0
14    508.0
13      NaN
15    190.0
16    410.0
17    510.0
Name: close, dtype: float64


,,first_date,last_days,max_close_ticker
sector,,,,
Semis,0,2024-01-01,2024-01-06,17
Tech,0,2024-01-01,2024-01-06,16


In [ ]:
###########################################################################################
fundamentals = pd.DataFrame({
    'ticker': ['AAPL','MSFT','NVDA'],
    'PE_ratio': [28.5, 32.1, 45.0],
    'MarketCap': [2.9e12, 3.2e12, 1.1e12]
})
# Now you can compute valuation metrics like daily volume as a percentage of market cap
df = df.merge(fundamentals, on='ticker', how='left')
# print(df.head(15))

# print(df.query('volume > 300 and sector == "Tech"'))
# print(df.eval('dollar_volume = close * volume',inplace=True))
df['sector'] = df['sector'].astype('category')
df_idx = df.set_index(['date','ticker']).sort_index()
days_values = df_idx.xs(pd.Timestamp('2024-01-03'),level='date')
# print(days_values)